In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as st
pd.set_option('display.max_columns', None)

import math

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_style('whitegrid')

import missingno as msno

from sklearn.preprocessing import StandardScaler
from scipy import stats



import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
#csv dosyamızı pandas aracılığı ile okuduk ve ilk 5 satırını yazdırdık.
df = pd.read_csv('/kaggle/input/videogamesales/vgsales.csv')
df.head()

In [ ]:
df.shape #toplamda 11 sütün ve 16598 satır bulunuyor.

In [ ]:
df.info #veri setimiz hakkında bilgi verir.



   * Rank - Toplam satışların sıralaması,integer türünde
   
   * Name - Oyunların ismi

   * Platform - Oyunun sürüm platformu (i.e. PC,PS4, etc.), object türünde

   * Year - Oyunun yayınlama yılı, float türünde

   * Genre - Oyunun türü ,object türünde

   * Publisher -Oyunun yayıncısı, object türünde

   * NA_Sales - Kuzey Amerikadaki satışlar , float türünde

   * EU_Sales - Avrupada'daki satışlar, float türünde

   * JP_Sales - Japonyadaki satışlar, float türünde

   * Other_Sales - Dünyanın geri kalanındaki satışlar, float türünde

   * Global_Sales - Toplam dünya çapındaki satışlar, float türünde

In [ ]:
 df.describe().T #sayısal veriler hakkındaki istatiksel değerler;

In [ ]:
df.isnull().sum()#hangi veride kaç tane eksik değer var?

## En çok hangi türde oyun var?

In [ ]:

df['Genre'].value_counts()

In [ ]:
plt.figure(figsize=(15, 10))
sns.countplot(x="Genre", data=df, order = df['Genre'].value_counts().index)
plt.xticks(rotation=90)

## En çok hangi yıl oyun çıktı?

In [ ]:
#Hangi yıl en çok oyun piyasaya sürüldü?
plt.figure(figsize=(15, 10))
sns.countplot(x ='Year',data=df,order = df.groupby(by=['Year'])['Name'].count().sort_values(ascending =False).index)
plt.xticks(rotation=100)

### Cevap
1. 2009.0  ->  1431
1. 2008.0  ->  1428
1. 2010.0  ->  1259
1. 2007.0  ->  1202
1. 2011.0  ->  1139

## Dünya çapında en yüksek satış yılı hangisi?

In [ ]:
df.year=df.groupby(by=['Year'])['Global_Sales'].sum()#Yıl ve dünya çapındaki değişkenleri gruplayık toplamlarını yazdırdık
df.year = df.year.reset_index()# iki değişkenin indexlerini eşitledik

In [ ]:
plt.figure(figsize=(15,10))
sns.barplot(x='Year', y ='Global_Sales',data= df.year)
plt.xticks(rotation=90)

## Cevap 
1. 2008.0 678.90
1. 2009.0 667.30
1. 2007.0 611.13
1. 2010.0 600.45
1. 2006.0 521.04

## Bir yıl içinde en çok hangi türde oyun piyasaya sürüldü?

In [ ]:
year_max_df = df.groupby(['Year', 'Genre']).size().reset_index(name='count')#Yıl ve tür değişkenine göre grupladık.
year_max_idx = year_max_df.groupby(['Year'])['count'].transform(max) == year_max_df['count']#aynı uzunlukta olmasını sağladık
year_max_genre = year_max_df[year_max_idx].reset_index(drop=True)
year_max_genre = year_max_genre.drop_duplicates(subset=["Year", "count"], keep='last').reset_index(drop=True)  #Yenilenen satırları kaldırdık.

In [ ]:
genre = year_max_genre['Genre'].values #Genre değişkenini numpy arrayine dönüştürdük

In [ ]:
plt.figure(figsize=(30, 15))
g = sns.barplot(x='Year', y='count', data=year_max_genre)
index = 0
for value in year_max_genre['count'].values:
#     print(asd)
    g.text(index, value + 5, str(genre[index] + '----' +str(value)), color='#000', size=14, rotation= 90, ha="center")
    index += 1




plt.xticks(rotation=90)
plt.show()

### *Cevap => 2009 Action ---> 139.36 milyon ve 2008 Action ---> 136.39 milyon.

## Dünya çapında en çok satış yapan platform hangisidir?

In [ ]:
data_platform = df.groupby(by=['Platform'])['Global_Sales'].sum()
data_platform = data_platform.reset_index()
data_platform = data_platform.sort_values(by=['Global_Sales'], ascending=False)

In [ ]:
plt.figure(figsize=(15, 10))
sns.barplot(x="Platform", y="Global_Sales", data=data_platform)
plt.xticks(rotation=90)

 ### Cevap --> PS2

## Hangi bireysel oyun dünya çapında en çok satışa sahip?

In [ ]:
top_game_sale = df.head(100) #Verimizin sadece ilk 100 örneği için araştırma yapacağız.
top_game_sale = top_game_sale[['Name', 'Year', 'Genre', 'Global_Sales']]#İsim,Yıl,Tür ve Dünya çapındaki değişkenlerimiz tanımladık.
top_game_sale = top_game_sale.sort_values(by=['Global_Sales'], ascending=False) #Bunları  Global_Sales bazında sıraladık.

In [ ]:
name = top_game_sale['Name']
year = top_game_sale['Year']
y = np.arange(0, 20)

In [ ]:
plt.figure(figsize=(30, 18))
g = sns.barplot(x='Name', y='Global_Sales', data=top_game_sale)
index = 0
for value in top_game_sale['Global_Sales']:
    g.text(index, value - 18, name[index], color='#000', size=14, rotation= 90, ha="center")
    index += 1

plt.xticks(y, top_game_sale['Year'], fontsize=14, rotation=90)
plt.xlabel('Release Year')
plt.show()

### Cevap ---> Wii Sports

## Türlere Göre Satışların Karşılaştırması

In [ ]:
comp_genre = df[['Genre', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales']] #Türün ve satışların oldugu sütünları çektik
# comp_genre
comp_map = comp_genre.groupby(by=['Genre']).sum() #ve bunlar Genre(Tür) degişkenine göre grupladık
# comp_map
comp_map

In [ ]:
#Heatmap yaparak satışlar ile türler arasındaki ilişkiyi görüyoruz.
#800 den aşşagı gidildikçe aralarında ilişkinin azaldığı görülür.
plt.figure(figsize=(15, 10))
sns.set(font_scale=1)
sns.heatmap(comp_map, annot=True, fmt = '.1f')

plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.show()

In [ ]:
comp_table = comp_map.reset_index()
comp_table = pd.melt(comp_table, id_vars=['Genre'], value_vars=['NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales'], var_name='Sale_Area', value_name='Sale_Price')
comp_table.head()

In [ ]:
plt.figure(figsize=(15, 10))
sns.barplot(x='Genre',y ='Sale_Price',hue='Sale_Area',data=comp_table)

#### Görülüğü üzere Action,Sports and Shooter en çok satış yapılan türler arasında ve Nort America tüm zamanların en çok satış yapan bölgesi.

## En İyi 20 Yayıncı


In [ ]:
#verimiznde bulunan Publisher ve Year değişkenlerini gruplayip indexlerini eşitliyoruz.
top_publisher = df.groupby(by=['Publisher'])['Year'].count().sort_values(ascending=False).head(20)
top_publisher = pd.DataFrame(top_publisher).reset_index()
top_publisher.head()

In [ ]:
plt.figure(figsize=(15, 10))
sns.countplot(x="Publisher", data=df, order = df.groupby(by=['Publisher'])['Year'].count().sort_values(ascending=False).iloc[:20].index)
plt.xticks(rotation=90)

#### Electronic Arts 1339 ile önde geliyor


## Bölgelere göre toplam gelir

In [ ]:
top_sale_reg = df[['NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales']]
top_sale_reg = top_sale_reg.sum().reset_index()
top_sale_reg = top_sale_reg.rename(columns={"index": "region", 0: "sale"})
top_sale_reg

In [ ]:
plt.figure(figsize=(12, 8))
sns.barplot(x='region', y='sale', data = top_sale_reg);

## Bölgelere Göre Satışlar(Pie Chart)

In [ ]:
labels = top_sale_reg['region']
sizes = top_sale_reg['sale']

In [ ]:
plt.figure(figsize=(10, 8))
plt.pie(sizes, labels=labels, autopct='%1.1f%%', shadow=True, startangle=90);